In [1]:
# Import dependencies
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'C:/Anaconda/webscrapes/chromedriver'}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('log-level=3')
browser = Browser('chrome', **executable_path, wait_time=3,headless=True,options=chrome_options)

In [3]:
# scrape links to each dealer and place in a list
url = 'https://www.lennox.com/locate/dealer-list/'
browser.visit(url)
html = browser.html
lennox_soup = BeautifulSoup(html, 'html.parser')

dealer_links = []

for i in lennox_soup.find_all(href=re.compile('/locate/dealer')):
    dealer_links.append('https://www.lennox.com'+i.get('href'))
dealer_links[0:5]

['https://www.lennox.comhttps://www.lennox.com/locate/dealer-list',
 'https://www.lennox.com/locate/dealer/al/semmes/action-htg-ac-and-refrig-inc',
 'https://www.lennox.com/locate/dealer/al/millbrook/air-now-heating-and-cooling-inc',
 'https://www.lennox.com/locate/dealer/al/huntsville/aire-serv-of-huntsville',
 'https://www.lennox.com/locate/dealer/al/birmingham/aireserv-over-the-mountain']

In [5]:
# import time
# start_time = time.time()

df = pd.DataFrame()
for dealer in dealer_links[1:10]:
    try:
        browser.visit(dealer)
        html = browser.html
        dealer_soup = BeautifulSoup(html, 'html.parser')
        
        name = dealer_soup.find(itemprop='name').get_text(strip=True)
        street = dealer_soup.find(itemprop='streetAddress').get_text(strip=True)
        city = dealer_soup.find(itemprop='addressLocality').get_text(strip=True)
        state = dealer_soup.find(itemprop='addressRegion').get_text(strip=True)
        zipcode = dealer_soup.find(itemprop='PostalCode').get_text(strip=True)
        country = dealer_soup.find(itemprop="addressCountry")['content']
        phone = dealer_soup.find(itemprop='telephone').get_text('[0-9]',strip=True)[-12:]
        lat = dealer_soup.find(itemprop="latitude")['content']
        long = dealer_soup.find(itemprop="longitude")['content']
        try:
            website = dealer_soup.find(attrs={'class':'website ng-binding ng-scope'}).get('href')
        except:
            website = ''
        
    except:
        pass
    dealer_df = {'name':name,
                 'street':street,
                 'city':city,
                 'state':state,
                 'zipcode':zipcode,
                 'country':country,
                 'phone':phone,
                 'lat':lat,
                 'long':long,
                 'website':website,
                 'brand':'Lennox'}

    df = df.append(dealer_df,ignore_index=True)
df
    # browser.quit()    
# print("--- %s seconds ---" % (time.time() - start_time))

,brand,city,country,lat,long,name,phone,state,street,website,zipcode
0,Lennox,SEMMES,United States,30.725487,-88.223907,Action Htg AC & Refrig Inc,251-645-0089,Alabama,7740 TARA DR,,36575-7478
1,Lennox,MILLBROOK,United States,32.480698,-86.363472,Air Now Heating & Cooling Inc,334-641-0368,Alabama,3481 MAIN ST,,36054-3216
2,Lennox,HUNTSVILLE,United States,34.699978,-86.591904,Aire Serv Of Huntsville,256-288-0164,Alabama,2106 W FERRY WAY SW STE E,,35801-5347
3,Lennox,BIRMINGHAM,United States,33.583153,-86.663979,Aireserv Over the Mountain,,Alabama,2812 RUFFNER RD,,35210-3916
4,Lennox,HUNTSVILLE,United States,34.800598,-86.533997,All Weather Htg & Air Cond Inc,256-677-4485,Alabama,104 DOMINION CIR,http://www.huntsvilleacexperts.com,35811-8924
5,Lennox,HUNTSVILLE,United States,34.746601,-86.597298,Annette Hale's Indoor Comfort Syste,256-885-0430,Alabama,1201 CHURCH ST NW,,35801-5917
6,Lennox,HEFLIN,United States,33.477699,-85.562599,Bain Htg & A/C Inc,855-230-2287,Alabama,1084 COUNTY ROAD 204,http://www.bainheatingandac.com,36264-8915
7,Lennox,Bay Minette,United States,30.8769,-87.781303,Bay Aire Inc,251-937-4676,Alabama,610 W. Railroad,http://www.bayaire.com,36507
8,Lennox,Dothan,United States,31.2285,-85.392197,Bob Woodall Air Care Systems,855-417-0696,Alabama,412 N Foster St,http://www.callbob.com/,36303


In [ ]:
from sqlalchemy import create_engine
import urllib


params = urllib.parse.quote_plus('DRIVER={SQL Server Native Client 10.0};Server=gmlsql2hou; DATABASE=invplannersapp;Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
df.to_sql('GBU_Webscrape',engine,if_exists='append',index=False)

In [17]:
print('Lennox Scrape completed')

Lennox Scrape completed
